In [1]:
import os
import pandas as pd
from pandas import DataFrame
from datetime import datetime as dt

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine, inspect

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Boolean 

from sqlalchemy.orm import Session

In [2]:
stations_file = os.path.join('../Data','closestStations.csv')
stations_df = pd.read_csv(stations_file, encoding="ISO-8859-1")
stations_df.head()

,ABBR,ID1,ID2,ID3,ID4,ID5
0,AZPS,723740-23194,722747-93084,722740-23160,723788-53135,723860-23169
1,AECI,724400-13995,724457-3938,723495-13987,724450-3945,724460-3947
2,AVA,727850-24157,727730-24153,727790-24146,726876-387,727720-24144
3,BANC,720614-205,724880-23185,724940-23234,723890-93193,726904-24231
4,BPAT,726876-387,727850-24157,727730-24153,726980-24229,725780-24156


In [3]:
closest_weather_file = os.path.join('../Data','2016-2018_Weather_all_stations_fixed.csv')
closet_weather_df = pd.read_csv(closest_weather_file, encoding="ISO-8859-1")
del closet_weather_df['Unnamed: 0']


In [4]:
closet_weather_df['ID'] = closet_weather_df['usaf'].astype(str) + '-' + closet_weather_df['wban'].astype(str)
closet_weather_df.head()

,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf,datetime,ID
0,723740,23194,340,21.0,99999,4.4,15.98,39.92,2016-01-01 00:00:00,723740-23194
1,723740,23194,300,21.0,22000,-2.8,10.94,26.96,2016-01-01 01:00:00,723740-23194
2,723740,23194,290,31.0,22000,-5.0,10.04,23.00,2016-01-01 02:00:00,723740-23194
3,723740,23194,280,31.0,22000,-3.3,12.02,26.06,2016-01-01 03:00:00,723740-23194
4,723740,23194,300,46.0,22000,-2.8,14.00,26.96,2016-01-01 04:00:00,723740-23194


In [5]:
final_avg_list = []
for index, row in stations_df.iterrows():
    name = row['ABBR']
    id1 = row['ID1']
    id2 = row['ID2']
    id3 = row['ID3']
    id4 = row['ID4']
    
    id1_df = closet_weather_df.loc[closet_weather_df['ID'] == id1]
    id2_df = closet_weather_df.loc[closet_weather_df['ID'] == id2]
    id3_df = closet_weather_df.loc[closet_weather_df['ID'] == id3]
    id4_df = closet_weather_df.loc[closet_weather_df['ID'] == id4]
    # pd.merge(pd.merge(df1,df2,on='name'),df3,on='name')
    if not(id1_df.empty) and not(id2_df.empty):
        merged_df= id1_df[['datetime','ID','tempf']].merge(id2_df[['datetime','ID','tempf']], on = 'datetime',how = 'inner')
        merged_df['avg'] = (merged_df['tempf_x'] + merged_df['tempf_y'])/2
        merged_df = merged_df.drop(columns=['ID_x','tempf_x','ID_y','tempf_y'])
    elif id1_df.empty and not(id2_df.empty):
        merged_df= id2_df
        merged_df = merged_df.rename(columns={'tempf':'avg'})
#         merged_df['avg'] = merged_df['tempf']
    elif not(id1_df.empty) and id2_df.empty:
        merged_df= id1_df
        merged_df = merged_df.rename(columns={'tempf':'avg'})  
    
    if not(id3_df.empty) and not(id4_df.empty):
        merged_df1= id3_df[['datetime','ID','tempf']].merge(id4_df[['datetime','ID','tempf']], on = 'datetime',how = 'inner')
        merged_df1['avg'] = (merged_df1['tempf_x'] + merged_df1['tempf_y'])/2
        merged_df1 = merged_df1.drop(columns=['ID_x','tempf_x','ID_y','tempf_y'])
    elif id3_df.empty and not(id4_df.empty):
        merged_df1= id4_df
        merged_df1 = merged_df1.rename(columns={'tempf':'avg'})
    elif not(id3_df.empty) and id4_df.empty:
        merged_df1= id3_df
        merged_df1 = merged_df1.rename(columns={'tempf':'avg'})
        
    final_df= merged_df[['datetime','avg']].merge(merged_df1[['datetime','avg']], on = 'datetime',how = 'inner')
    final_df['avg'] = (final_df['avg_x'] + final_df['avg_y'])/2
    final_df = final_df.drop(columns=['avg_x','avg_y'])
    final_df['abbr'] = name
    final_avg_list.append(final_df)
#     break
# print(len(final_avg_list))
final_avg_df = pd.concat(final_avg_list)
final_avg_df['datetime'] = pd.to_datetime(final_avg_df['datetime'])
final_avg_df.head()
# final_avg_df.to_csv('testing.csv')



,datetime,avg,abbr
0,2016-01-01 01:00:00,44.240,AZPS
1,2016-01-01 02:00:00,41.495,AZPS
2,2016-01-01 03:00:00,41.495,AZPS
3,2016-01-01 04:00:00,41.045,AZPS
4,2016-01-01 05:00:00,39.290,AZPS


In [6]:
engine = create_engine("sqlite:///../Data/megawatt.sqlite")

In [7]:
sql_df = DataFrame(engine.execute("SELECT * FROM demand where date >'2016-01-01'and date < '2018-12-01'").fetchall())
sql_df = sql_df.rename(columns={
    sql_df.columns[0]: 'datetime',
    sql_df.columns[1]: 'demand',
    sql_df.columns[2]: 'abbr',
    sql_df.columns[3] : 'datetime_utc',
    sql_df.columns[4] : 'date',
    sql_df.columns[5] : 'hour'
    })

sql_df['datetime'] = pd.to_datetime(sql_df['datetime'])
sql_df['datetime_utc'] = pd.to_datetime(sql_df['datetime_utc'])
# newdf.sort_values('datetime',ascending=True)
sql_df.head()


,datetime,demand,abbr,datetime_utc,date,hour
0,2018-10-20 17:00:00,3707.0,AZPS,2018-10-21 00:00:00,2018-10-20,17
1,2018-10-20 16:00:00,3586.0,AZPS,2018-10-20 23:00:00,2018-10-20,16
2,2018-10-20 15:00:00,3395.0,AZPS,2018-10-20 22:00:00,2018-10-20,15
3,2018-10-20 14:00:00,3241.0,AZPS,2018-10-20 21:00:00,2018-10-20,14
4,2018-10-20 13:00:00,3041.0,AZPS,2018-10-20 20:00:00,2018-10-20,13


In [8]:
merged_df = pd.merge(sql_df, final_avg_df, on=['datetime','abbr'],how='inner')
merged_df.to_csv('final_merged.csv')
merged_df.head()

,datetime,demand,abbr,datetime_utc,date,hour,avg
0,2018-10-20 17:00:00,3707.0,AZPS,2018-10-21 00:00:00,2018-10-20,17,69.260
1,2018-10-20 16:00:00,3586.0,AZPS,2018-10-20 23:00:00,2018-10-20,16,67.055
2,2018-10-20 15:00:00,3395.0,AZPS,2018-10-20 22:00:00,2018-10-20,15,64.130
3,2018-10-20 01:00:00,2608.0,AZPS,2018-10-20 08:00:00,2018-10-20,1,71.285
4,2018-10-20 00:00:00,2737.0,AZPS,2018-10-20 07:00:00,2018-10-20,0,74.570


In [10]:
sample_df = merged_df.loc[merged_df['abbr'] == 'AZPS']
sample_df.to_csv('sample_merged.csv')
sample_df.head()


,datetime,demand,abbr,datetime_utc,date,hour,avg
0,2018-10-20 17:00:00,3707.0,AZPS,2018-10-21 00:00:00,2018-10-20,17,69.260
1,2018-10-20 16:00:00,3586.0,AZPS,2018-10-20 23:00:00,2018-10-20,16,67.055
2,2018-10-20 15:00:00,3395.0,AZPS,2018-10-20 22:00:00,2018-10-20,15,64.130
3,2018-10-20 01:00:00,2608.0,AZPS,2018-10-20 08:00:00,2018-10-20,1,71.285
4,2018-10-20 00:00:00,2737.0,AZPS,2018-10-20 07:00:00,2018-10-20,0,74.570


In [11]:
# run a sample to validate data
# 0	AZPS	723740-23194	722747-93084	722740-23160	723788-53135	723860-23169

In [12]:
name = 'AZPS'
test_id1_df = closet_weather_df.loc[closet_weather_df['ID'] == '723740-23194']
test_id2_df = closet_weather_df.loc[closet_weather_df['ID'] == '722747-93084']
test_id3_df = closet_weather_df.loc[closet_weather_df['ID'] == '722740-23160']
test_id4_df = closet_weather_df.loc[closet_weather_df['ID'] == '723788-53135']
test_id5_df = closet_weather_df.loc[closet_weather_df['ID'] == '723860-23169']

# pd.merge(pd.merge(df1,df2,on='name'),df3,on='name')

test_merged_df= test_id1_df[['datetime','ID','tempf']].merge(test_id2_df[['datetime','ID','tempf']], on = 'datetime',how = 'inner')
test_merged_df['avg'] = (test_merged_df['tempf_x'] + test_merged_df['tempf_y'])/2
test_merged_df = test_merged_df.drop(columns=['ID_x','tempf_x','ID_y','tempf_y'])

test_merged_df1= test_id3_df[['datetime','ID','tempf']].merge(test_id4_df[['datetime','ID','tempf']], on = 'datetime',how = 'inner')
test_merged_df1['avg'] = (test_merged_df1['tempf_x'] + test_merged_df1['tempf_y'])/2
test_merged_df1 = test_merged_df1.drop(columns=['ID_x','tempf_x','ID_y','tempf_y'])

test_final_df= test_merged_df[['datetime','avg']].merge(test_merged_df1[['datetime','avg']], on = 'datetime',how = 'inner')
test_final_df['avg'] = (test_final_df['avg_x'] + test_final_df['avg_y'])/2
test_final_df = test_final_df.drop(columns=['avg_x','avg_y'])
test_final_df['abbr'] = name
test_final_df['datetime'] = pd.to_datetime(test_final_df['datetime'])

# merged_df= id1_df[['datetime','ID','tempf']].merge(id2_df[['datetime','ID','tempf']], on = 'datetime',how = 'inner')
test_final_df.to_csv('test_sample_merged.csv')
